In [2]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from datetime import datetime
from django.utils import timezone
from bookings.models import Room, RoomType, Staff, Booking

# 1. CRUD operations

1.1 ให้เพิ่มห้อง (`Room`) โดยมีข้อมูลดังนี้ (0.5 คะแนน)

- number: 203
- name: Laboratory A
- capacity: 50
- description: Laboratory for Web Development
- room_types: Lecture, Office

In [5]:
# CODE HERE

lecture_room = RoomType.objects.get(name="Lecture")
office_room = RoomType.objects.get(name="Office")

new_room = Room(
    number=203,
    name="Laboratory A",
    capacity=50,
    description="Laboratory for Web Development"
)

new_room.save()
new_room.room_types.add(lecture_room)
new_room.room_types.add(office_room)



In [8]:
# CHECK RESULTS (ห้ามลบ/แก้ไข)
r = Room.objects.get(number="203")
for t in r.room_types.all():
    print(f"ROOM: {r}; ROOM TYPE: {t.name}")

ROOM: Room 203: Laboratory A; ROOM TYPE: Lecture
ROOM: Room 203: Laboratory A; ROOM TYPE: Office


1.2 เนื่องจากนโยบายการจองคือห้ามจองห้องเกิน 1 ชั่วโมง ดังนั้นให้ค้นหาการจองห้อง (`Booking`) ที่มีระยะเวลามากกว่า 1 ชั่วโมงและทำการลบออกทั้งหมด (1 คะแนน)

**Hint: ให้ annotate() คำนวณระยะเวลาการจอง และนำระยะเวลาการจองมา filter() ดังตัวอย่าง**

```python
from datetime import timedelta

queryset.filter(
    duration__gt=timedelta(hours=1)
)
```

In [26]:
# CODE HERE - Filter การจองที่เกิน 1 ชม.
from datetime import timedelta
from django.db.models import F

rooms = Booking.objects.annotate(durations=F('end_time')-F('start_time'))
invalid_room = rooms.filter(durations__gt=timedelta(hours=1))

# print(*invalid_room, sep='\n')
# print(invalid_room.count())

In [24]:
# CHECK RESULTS - จำนวนการจองทั้งหมด (ห้ามลบ/แก้ไข)
Booking.objects.count()

33

In [29]:
# CODE HERE - ลบการจองที่เกิน 1 ชม.
for room in invalid_room:
    room.delete()

In [30]:
# CHECK RESULTS - จำนวนการจองทั้งหมดหลังการลบ (ห้ามลบ/แก้ไข)
Booking.objects.count()

20

1.3 ทำการเพิ่มการจอง (`Booking`) ซึ่งมีรายละเอียดดังนี้ (1 คะแนน)

- staff: "Jane Smith"
- room: "Lecture Hall A"
- start_time: 26/08/2024 13:00
- end_time: 26/08/2024 14:00
- purpose: "To teach subject: IT Project Management"

In [11]:
# CODE HERE
from zoneinfo import ZoneInfo
from django.utils import timezone

angelina_jolie = Staff.objects.get(name="Jane Smith")
sooksan_lecture = Room.objects.get(name="Lecture Hall A")

start_time=datetime(2024, 8, 26, 13, 0 ,0, tzinfo = ZoneInfo(key='Asia/Bangkok'))
end_time=datetime(2024, 8, 26, 14, 0 , 0, tzinfo = ZoneInfo(key='Asia/Bangkok'))

new_booking = Booking(
    staff=angelina_jolie,
    room=sooksan_lecture,
    start_time=start_time,
    end_time=end_time,
    purpose="To teach subject: IT Project Management"
)

new_booking.save()

In [12]:
# CHECK RESULTS (ห้ามลบ/แก้ไข)
booking = Booking.objects.last()
print(f"STAFF: {booking.staff}, ROOM: {booking.room}, START: {timezone.localtime(booking.start_time)} - {timezone.localtime(booking.end_time)}")

STAFF: Jane Smith, ROOM: Room 202: Lecture Hall A, START: 2024-08-26 13:00:00+07:00 - 2024-08-26 14:00:00+07:00


1.4 จากข้อ 1.3 คุณ "Jane Smith" จองห้องผิดจริงๆ เขาอยากจะจองเป็น "Laboratory A" (ที่สร้างในข้อ 1.1) (0.5 คะแนน)

In [13]:
lab_a = Room.objects.get(name="Laboratory A")
new_booking.room = lab_a
new_booking.save()

In [14]:
# CHECK RESULTS (ห้ามลบ/แก้ไข)
booking = Booking.objects.last()
print(f"STAFF: {booking.staff}, ROOM: {booking.room}, START: {timezone.localtime(booking.start_time)} - {timezone.localtime(booking.end_time)}")

STAFF: Jane Smith, ROOM: Room 203: Laboratory A, START: 2024-08-26 13:00:00+07:00 - 2024-08-26 14:00:00+07:00


# 2. Making Queries

**ให้ทำการ reset DB และ import ข้อมูลใน bookings.sql เข้าไปใหม่**

2.1 ให้หาจำนวนการจองที่จองในช่วงวันที่ (`Booking`) ในช่วงวันที่ 2024-08-18 ถึง 2024-08-25 ที่จองห้องในประเภท (`RoomType`) "Conference" (1 คะแนน)

**Expected Output**

จะต้องมี 7 รายการ

*หมายเหตุ: จะต้อง get ข้อมูลมาโดยใช้การ query ด้วยชื่อตามที่โจทย์ว่าเท่านั้น ห้ามใช้ id ใน database*

In [32]:
# CODE HERE
room = RoomType.objects.get(name="Conference")

result = Booking.objects.filter(
    start_time__year=2024,
    start_time__month=8,
    start_time__day__range=(18, 25),
    room__room_types=room
)

print(*result, sep='\n\n')

Booking by John Doe for Room 101: Main Conference Room from 2024-08-18 09:00:00+00:00 to 2024-08-18 11:00:00+00:00

Booking by Emily Davis for Room 303: Executive Office from 2024-08-18 15:00:00+00:00 to 2024-08-18 16:00:00+00:00

Booking by John Doe for Room 101: Main Conference Room from 2024-08-19 10:00:00+00:00 to 2024-08-19 12:00:00+00:00

Booking by Jane Smith for Room 303: Executive Office from 2024-08-19 15:00:00+00:00 to 2024-08-19 16:00:00+00:00

Booking by Emily Davis for Room 303: Executive Office from 2024-08-22 14:00:00+00:00 to 2024-08-22 15:30:00+00:00

Booking by John Doe for Room 303: Executive Office from 2024-08-24 11:00:00+00:00 to 2024-08-24 13:00:00+00:00

Booking by Jane Smith for Room 101: Main Conference Room from 2024-08-24 09:30:00+00:00 to 2024-08-24 10:30:00+00:00


2.2 ให้หาจำนวนการจองของห้องแต่ละห้อง โดยแสดงผลดังนี้ - เรียงจากจำนวนการจอง มาก ไป น้อย (1 คะแนน)

**Expected Output**

```
ROOM: Room 101: Main Conference Room, BOOKINGS: 12
ROOM: Room 303: Executive Office, BOOKINGS: 12
ROOM: Room 202: Lecture Hall A, BOOKINGS: 9
```

In [47]:
# CODE HERE
from django.db.models import Count

room_bookings = Room.objects.annotate(bookings=Count('booking')).order_by('-bookings')

for i in room_bookings:
    print(f"ROOM: Room {i.number}: {i.name}, BOOKINGS: {i.bookings}")

ROOM: Room 101: Main Conference Room, BOOKINGS: 12
ROOM: Room 303: Executive Office, BOOKINGS: 12
ROOM: Room 202: Lecture Hall A, BOOKINGS: 9


2.3 ในช่วงวันที่ 2024-08-18 ถึง 2024-08-25 ให้หาว่าพนักงานแต่ละคนมีการจองห้องไหนบ้าง โดยแสดงผลดังนี้ (2 คะแนน)

**Expected Output**

```
STAFF: John Doe, BOOKED ROOMS: Main Conference Room, Executive Office
STAFF: Jane Smith, BOOKED ROOMS: Main Conference Room, Executive Office, Lecture Hall A
STAFF: Emily Davis, BOOKED ROOMS: Lecture Hall A, Executive Office
```

In [108]:
# CODE HERE
# all_bookings = Booking.objects.values('staff', 'room').distinct()
# all_bookings = Staff.objects.values('id', 'name', 'booking__room__room_types__name').distinct().order_by('id')
all_staff = Staff.objects.all().values('id')

all_staff_bookings = {}
for i in all_staff:
    temp_list = []
    staff_bookings = Staff.objects.filter(
        id=i['id'],
        booking__start_time__year=2024,
        booking__start_time__month=8,
        booking__start_time__day__range=(18, 25)
    ).values('name', 'booking__room__name').distinct()
    for room in staff_bookings:
        temp_list.append(room['booking__room__name'])
    all_staff_bookings[i['id']] = temp_list
    
    print(f"STAFF: {staff_bookings[0]['name']}, BOOKED ROOMS: {', '.join(temp_list)}")    


STAFF: John Doe, BOOKED ROOMS: Executive Office, Main Conference Room
STAFF: Jane Smith, BOOKED ROOMS: Executive Office, Lecture Hall A, Main Conference Room
STAFF: Emily Davis, BOOKED ROOMS: Executive Office, Lecture Hall A


2.4 ให้หาว่าการจองห้องในแต่ละวันมีค่าความจุ (ดูจาก `Room.capacity`) โดยรวมเท่าไหร่ เรียงลำดับความจุรวมจากมากไปน้อย (2 คะแนน)

**Expected Output**

```
DATE: 2024-08-26, SUM CAPACITY: 155
DATE: 2024-08-24, SUM CAPACITY: 155
DATE: 2024-08-30, SUM CAPACITY: 155
DATE: 2024-08-19, SUM CAPACITY: 155
DATE: 2024-09-02, SUM CAPACITY: 100
DATE: 2024-09-06, SUM CAPACITY: 100
DATE: 2024-09-12, SUM CAPACITY: 100
DATE: 2024-09-07, SUM CAPACITY: 100
DATE: 2024-09-13, SUM CAPACITY: 100
DATE: 2024-09-14, SUM CAPACITY: 55
DATE: 2024-08-18, SUM CAPACITY: 55
DATE: 2024-08-28, SUM CAPACITY: 55
DATE: 2024-09-05, SUM CAPACITY: 50
DATE: 2024-09-08, SUM CAPACITY: 50
DATE: 2024-09-01, SUM CAPACITY: 50
DATE: 2024-09-10, SUM CAPACITY: 50
DATE: 2024-09-03, SUM CAPACITY: 50
DATE: 2024-09-11, SUM CAPACITY: 5
DATE: 2024-08-27, SUM CAPACITY: 5
DATE: 2024-09-09, SUM CAPACITY: 5
DATE: 2024-09-04, SUM CAPACITY: 5
DATE: 2024-08-22, SUM CAPACITY: 5
```

In [201]:
# CODE HERE
from django.db.models import Sum, Value
from django.db.models.functions import Concat

dash_values = Value('-')

day_bookings = Booking.objects.all()

all_date = set()
for i in day_bookings:
    all_date.add(i.start_time.date())

for date in all_date:
    sum_booking = Booking.objects.filter(
        start_time__year=date.year,
        start_time__month=date.month,
        start_time__day=date.day
    ).annotate(capacity=Sum('room__capacity')).values('capacity')
    print(sum_booking[0]['capacity'])

50
50
100
100
5
50
100
5
50
50
5
5
100
100
5
5
50
5
100
100
50
5


2.5 จงหาการจองที่มีวัตถุประสงค์ (`Booking.purpose`) ที่มีความยาวตัวอักษรมากกว่า 20 ตัวอักษร หรือ จะต้องใช้ห้องที่มีความจุมากกว่าหรือเท่ากับ 100 (1 คะแนน)

**Expected Output**

```
START DATE: 2024-08-18, ROOM: Room 101: Main Conference Room, PURPOSE LENGTH: 21, ROOM CAP: 50
START DATE: 2024-08-19, ROOM: Room 202: Lecture Hall A, PURPOSE LENGTH: 19, ROOM CAP: 100
START DATE: 2024-08-19, ROOM: Room 303: Executive Office, PURPOSE LENGTH: 22, ROOM CAP: 5
START DATE: 2024-08-24, ROOM: Room 303: Executive Office, PURPOSE LENGTH: 25, ROOM CAP: 5
START DATE: 2024-08-24, ROOM: Room 202: Lecture Hall A, PURPOSE LENGTH: 15, ROOM CAP: 100
START DATE: 2024-08-26, ROOM: Room 202: Lecture Hall A, PURPOSE LENGTH: 12, ROOM CAP: 100
START DATE: 2024-08-27, ROOM: Room 303: Executive Office, PURPOSE LENGTH: 21, ROOM CAP: 5
START DATE: 2024-08-30, ROOM: Room 202: Lecture Hall A, PURPOSE LENGTH: 19, ROOM CAP: 100
START DATE: 2024-08-30, ROOM: Room 303: Executive Office, PURPOSE LENGTH: 23, ROOM CAP: 5
START DATE: 2024-09-02, ROOM: Room 202: Lecture Hall A, PURPOSE LENGTH: 21, ROOM CAP: 100
START DATE: 2024-09-03, ROOM: Room 101: Main Conference Room, PURPOSE LENGTH: 24, ROOM CAP: 50
START DATE: 2024-09-05, ROOM: Room 101: Main Conference Room, PURPOSE LENGTH: 23, ROOM CAP: 50
START DATE: 2024-09-06, ROOM: Room 202: Lecture Hall A, PURPOSE LENGTH: 22, ROOM CAP: 100
START DATE: 2024-09-07, ROOM: Room 202: Lecture Hall A, PURPOSE LENGTH: 18, ROOM CAP: 100
START DATE: 2024-09-10, ROOM: Room 101: Main Conference Room, PURPOSE LENGTH: 22, ROOM CAP: 50
START DATE: 2024-09-12, ROOM: Room 202: Lecture Hall A, PURPOSE LENGTH: 22, ROOM CAP: 100
START DATE: 2024-09-13, ROOM: Room 202: Lecture Hall A, PURPOSE LENGTH: 14, ROOM CAP: 100
START DATE: 2024-09-14, ROOM: Room 101: Main Conference Room, PURPOSE LENGTH: 21, ROOM CAP: 50
```

In [ ]:
# CODE HERE
